### Chroma DB

- An open source, embedding-optimized vector database.

- In-memory or persistent storage options. REST API and Python support.

- Filtering feature (ability to search with metadata). Easy to install, lightweight and fast.

- Integrates with tools like LangChain, LlamaIndex. Free and suitable for local use.

In [2]:
# Building a sample vectordb
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [3]:
loader = TextLoader('speech.txt')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 30)
docs = text_splitter.split_documents(documents)

In [4]:
embeddings = (
    OllamaEmbeddings(model="nomic-embed-text") # A high-performing open embedding model with a large token context window. 
)

vectordb = Chroma.from_documents(documents = docs, embedding = embeddings)
vectordb

C:\Users\furka\AppData\Local\Temp\ipykernel_5392\43834663.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  OllamaEmbeddings(model="nomic-embed-text") # A high-performing open embedding model with a large token context window.


In [ ]:
# Query 
query = "How does the speaker describe the desired outcome of the war?"
docs = vectordb.similarity_search(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

In [6]:
# Saving to the disk
vectordb = Chroma.from_documents(documents = docs, embedding = embeddings, persist_directory="./chroma_db")

In [7]:
# load the disk
new_db = Chroma(persist_directory="./chroma_db", embedding_function = embeddings)
docs = new_db.similarity_search(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

In [8]:
docs = new_db.similarity_search_with_score(query)
docs

[(Document(id='6db6a571-bc01-486a-8a69-4ebc443ad08d', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  343.21935573938595),
 (Document(id='17ea0a8b-04e9-4dc9-ab38-a96ae867916b', metadata={'source': 'spee

##### Retriever

In [12]:
retriever = vectordb.as_retriever()
retriever.invoke(query)[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'